###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [28]:
import pandas as pd

###Archivo i7h7-wmjc.csv

Contiene información sobre casos de hurto a entidades financieras. Delito definido como: Acciones delincuenciales mediante las cuales, ejerciendo violencia sobre las personas o las cosas, con penetración o permanencia arbitraria, engañosa o clandestina y violando o superando seguridades electrónicas o de otro tipo, proceden a apoderarse del dinero de una entidad financiera. (art. 239 Hurto. según el Código Penal Colombiano, Ley 599 de 2000, se define como “el que se apodere de una cosa mueble ajena, con el propósito de obtener provecho para sí o para otro”.).

In [29]:
df = pd.read_csv("https://www.datos.gov.co/resource/i7h7-wmjc.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,cantidad
0,2024-07-23T00:00:00.000,63,QUINDIO,63001,ARMENIA,1
1,2024-07-22T00:00:00.000,8,ATLANTICO,08606,REPELON,1
2,2024-07-08T00:00:00.000,13,BOLIVAR,13468,SANTA CRUZ DE MOMPOX,1
3,2024-06-19T00:00:00.000,47,MAGDALENA,47001,SANTA MARTA,1
4,2024-06-14T00:00:00.000,13,BOLIVAR,13647,SAN ESTANISLAO,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2442 entries, 0 to 2441
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   fecha_hecho   2442 non-null   object
 1   cod_depto     2442 non-null   int64 
 2   departamento  2442 non-null   object
 3   cod_muni      2442 non-null   object
 4   municipio     2442 non-null   object
 5   cantidad      2442 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 114.6+ KB


### Eliminación de columnas irrelevantes para el proyecto

In [31]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad']
theft_from_financial_institutions_MinDefensa = df[relevant_cols]
theft_from_financial_institutions_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2442 entries, 0 to 2441
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_muni      2442 non-null   object
 1   departamento  2442 non-null   object
 2   municipio     2442 non-null   object
 3   fecha_hecho   2442 non-null   object
 4   cantidad      2442 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 95.5+ KB


### Convertir columna fecha_hecho a tipo date

In [32]:
# Hacer una copia explícita del DataFrame
theft_from_financial_institutions_MinDefensa_ = theft_from_financial_institutions_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
theft_from_financial_institutions_MinDefensa_['fecha_hecho'] = pd.to_datetime(theft_from_financial_institutions_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
theft_from_financial_institutions_MinDefensa_['fecha_hecho'] = theft_from_financial_institutions_MinDefensa_['fecha_hecho'].dt.year

In [33]:
theft_from_financial_institutions_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003],
      dtype=int32)

### Verificar valores nulos

In [34]:
theft_from_financial_institutions_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,0
municipio,0
fecha_hecho,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [35]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(theft_from_financial_institutions_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['QUINDIO' 'ATLANTICO' 'BOLIVAR' 'MAGDALENA' 'HUILA' 'CUNDINAMARCA' 'META'
 'BOGOTA D.C.' 'VALLE DEL CAUCA' 'LA GUAJIRA' 'CASANARE' 'TOLIMA'
 'CORDOBA' 'ANTIOQUIA' 'NORTE DE SANTANDER' 'RISARALDA' 'CAQUETA'
 'SANTANDER' 'SUCRE' 'PUTUMAYO' 'CAUCA' 'CESAR' 'NARIÑO' 'BOYACA' 'CALDAS'
 'GUAINIA' 'SAN ANDRES ISLAS' 'CHOCO' 'GUAVIARE' 'AMAZONAS' 'VAUPES'
 'ARAUCA']

Categorías en la columna 'municipio':
['ARMENIA' 'REPELON' 'SANTA CRUZ DE MOMPOX' 'SANTA MARTA' 'SAN ESTANISLAO'
 'CIENAGA' 'SANTO TOMAS' 'CAMPOALEGRE' 'VILLA DE SAN DIEGO DE UBATE'
 'BARRANQUILLA' 'VILLAVICENCIO' 'BOGOTA D.C.' 'PUERTO COLOMBIA' 'CALI'
 'GUAMAL' 'CARTAGO' 'FUNZA' 'PALMIRA' 'RIOHACHA' 'SOLEDAD' 'NEIVA' 'PORE'
 'ESPINAL' 'CAJICA' 'SOACHA' 'MONTERIA' 'ENVIGADO' 'CACHIRA' 'EL COLEGIO'
 'CALARCA' 'MEDELLIN' 'PEREIRA' 'FLORENCIA' 'BARBOSA' 'COYAIMA' 'TOCAIMA'
 'SANTIAGO DE TOLU' 'TENJO' 'GIRARDOT' 'CANDELARIA' 'PUERTO ASIS'
 'FUSAGASUGA' 'CAMPO DE LA CRUZ' 'COROZAL' 'PADILLA' 'E

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [36]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [37]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    theft_from_financial_institutions_MinDefensa_[col] = theft_from_financial_institutions_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [38]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df theft_from_financial_institutions_MinDefensa_

In [39]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
theft_from_financial_institutions_MinDefensa_['cod_muni'] = theft_from_financial_institutions_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = theft_from_financial_institutions_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [40]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [41]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,2442,1.0


In [42]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = theft_from_financial_institutions_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    63001
1    08606
2    13468
3    47001
4    13647



Nota: el 100%  de los codigos que identifican a los municipios en el df theft_from_financial_institutions_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [43]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [44]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [45]:
# Comparar listas de departamento
compare_lists(theft_from_financial_institutions_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en theft_from_financial_institutions_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en theft_from_financial_institutions_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'SAN ANDRES ISLAS'}


  - Solucionar discrepancias en nombres de departamento

In [46]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES ISLAS': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe theft_from_financial_institutions_MinDefensa_
theft_from_financial_institutions_MinDefensa_['departamento'] = theft_from_financial_institutions_MinDefensa_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [47]:
# Comparar listas de departamento
compare_lists(theft_from_financial_institutions_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en theft_from_financial_institutions_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en theft_from_financial_institutions_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [48]:
# Comparar listas de códigos
compare_lists(theft_from_financial_institutions_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en theft_from_financial_institutions_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en theft_from_financial_institutions_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [49]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_theft_from_financial_institutions_MinDefensa = theft_from_financial_institutions_MinDefensa_.drop(columns=columns_to_drop)

In [50]:
# Adicionar columna para trazabilidad de la fuente
final_theft_from_financial_institutions_MinDefensa['source_id'] = 26

In [51]:
final_theft_from_financial_institutions_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'cantidad', 'source_id'], dtype='object')

In [52]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_theft_from_financial_institutions_MinDefensa.rename(columns=translation_map, inplace=True)

In [53]:
#Estructura final del dataset a integrar a la base de datos
final_theft_from_financial_institutions_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2442 entries, 0 to 2441
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dane_code         2442 non-null   object
 1   year_of_incident  2442 non-null   int32 
 2   amount            2442 non-null   int64 
 3   source_id         2442 non-null   int64 
dtypes: int32(1), int64(2), object(1)
memory usage: 66.9+ KB


## Salvar en archivo csv en el drive

In [54]:
# Guardar en archivos CSV en el drive
final_theft_from_financial_institutions_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/theft_from_financial_institutions_MinDefensa.csv', index=False)